**About** : This notebook is used to infer models.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

### Imports

In [ ]:
import os
import gc
import sys
import cv2
import glob
import json
import torch
import shutil
import joblib
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import expit
from joblib import Parallel, delayed

warnings.simplefilter(action="ignore", category=UserWarning)
pd.set_option('display.max_columns', 100)
pd.set_option("display.precision", 3)

In [ ]:
from util.logger import Config
from util.metrics import macro_auc
from util.torch import load_model_weights

from data.preparation import prepare_data, prepare_folds
from model_zoo.models import define_model
from inference.predict import infer_onnx, load_sample, infer_sample
from params import CLASSES

### Params

In [ ]:
EVAL = False

In [ ]:
if EVAL:
    DATA_PATH = "../input/train_audio/"
else:
    DATA_PATH = "../input/unlabeled_soundscapes/"

In [ ]:
BATCH_SIZE = 48
USE_FP16 = True
NUM_WORKERS = 4

DEVICE = "cuda"

DURATION = 5
SR = 32000

In [ ]:
FOLD = 0 if EVAL else "fullfit_0"

EXP_FOLDERS = [
 
    # ("../logs/2024-05-12/1/", [f"fullfit_{i}" for i in range(5)], "torch"),  # effnet-b0 PL0.72
    # ("../logs/2024-05-12/2/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b1 PL0.72
    # ("../logs/2024-05-13/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PLBirdnet
    # ("../logs/2024-05-13/3/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL0.72 cpmp params
    # ("../logs/2024-05-14/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL0.72 cpmp params default head  <- LB 0.7

    # # ROUND 1
    # ("../logs/2024-05-14/22/", [f"{i}" for i in range(4)], "torch"),  # vit-b0 PL all above             0.69
    # ("../logs/2024-05-15/0/", [f"{i}" for i in range(4)], "torch"),   # vit-b0 PL all above more augs   0.65
    # ("../logs/2024-05-15/1/", [f"{i}" for i in range(4)], "torch"),   # vit-b0 PL all above no augs     0.70  <--
    # ("../logs/2024-05-15/2/", [f"{i}" for i in range(4)], "torch"),   # vit-b0 PL all above fix mix     0.69

    # ("../logs/2024-05-16/1/", [f"{i}" for i in range(4)], "torch"),  # vit-b0 PL all above less augs

    # # ROUND 0 - Mixup only
    # ("../logs/2024-05-16/2/", [f"{i}" for i in range(4)], "torch"),   # vit-b0
    # ("../logs/2024-05-16/3/", [f"{i}" for i in range(4)], "torch"),   # vit-b1
    # ("../logs/2024-05-16/4/", [f"{i}" for i in range(4)], "torch"),   # mixnet
    # ("../logs/2024-05-16/5/", [f"{i}" for i in range(4)], "torch"),   # mobilenet
    # ("../logs/2024-05-16/6/", [f"{i}" for i in range(4)], "torch"),   # mnasnet
    # ("../logs/2024-05-16/7/", [f"{i}" for i in range(4)], "torch"),   # b0
    # ("../logs/2024-05-16/8/", [f"{i}" for i in range(4)], "torch"),   # tinynet
    # ("../logs/2024-05-16/9/", [f"{i}" for i in range(4)], "torch"),   # b0-v2

    # ROUND 1 - 
    ("../logs/2024-05-16/12/", [f"{i}" for i in range(4)], "torch"),   # vit-b0 PL2 no augs             0.71
    # ("../logs/2024-05-16/14/", [f"{i}" for i in range(4)], "torch"),   # vit-b0 PL2 no overfitnoisy   0.71

    # ("../logs/2024-05-17/0/", [f"{i}" for i in range(4)], "torch"),   # v2-b0 PL2 no augs     
    # ("../logs/2024-05-17/1/", [f"{i}" for i in range(4)], "torch"),   # mixnet PL2 no augs     
    ("../logs/2024-05-17/2/", [f"{i}" for i in range(4)], "torch"),   # mobilenetv2 PL2 no augs         0.69
    ("../logs/2024-05-17/3/", [f"{i}" for i in range(4)], "torch"),   # mnasnet PL2 no augs             0.70

    # ("../logs/2024-05-17/4/", [f"{i}" for i in range(4)], "torch"),   # b0 PL2 no augs
    # ("../logs/2024-05-17/5/", [f"{i}" for i in range(4)], "torch"),   # tinynet PL2 no augs
    # ("../logs/2024-05-17/8/", [f"{i}" for i in range(4)], "torch"),   # effvitm0 PL2 no augs
    # ("../logs/2024-05-17/9/", [f"{i}" for i in range(4)], "torch"),   # effvitm1 PL2 no augs
    ("../logs/2024-05-18/0/", [f"{i}" for i in range(4)], "torch"),   # effvitm2 PL2 no augs            0.69
    ("../logs/2024-05-18/1/", [f"{i}" for i in range(4)], "torch"),   # effvitm3 PL2 no augs            0.70
    # ("../logs/2024-05-18/4/", [f"{i}" for i in range(4)], "torch"),   # mnasnet PL2 no augs 20 eps    0.67
    ("../logs/2024-05-18/8/", [f"{i}" for i in range(4)], "torch"),   # mobilenetv3_s PL2 no augs       0.??
    ("../logs/2024-05-19/0/", [f"{i}" for i in range(4)], "torch"),   # efficientnet_lite0 PL2 no augs  0.70
    ("../logs/2024-05-19/1/", [f"{i}" for i in range(4)], "torch"),   # effvitm4 PL2 no augs            0.??
    # ("../logs/2024-05-19/2/", [f"{i}" for i in range(4)], "torch"),   # effvitm5 PL2 no augs            0.69

    # ("../logs/2024-05-19/6/", [f"{i}" for i in range(4)], "torch"),   # effvitm3 PL2 no augs 16/48 pl   0.70

    ("../logs/2024-05-19/9/", [f"{i}" for i in range(4)], "torch"),   # regnety_002 PL2 no augs
    ("../logs/2024-05-19/10/", [f"{i}" for i in range(4)], "torch"),  # regnety_004 PL2 no augs
    ("../logs/2024-05-20/1/", [f"{i}" for i in range(4)], "torch"),   # lcnet_100 PL2 no augs
    ("../logs/2024-05-20/2/", [f"{i}" for i in range(4)], "torch"),   # mobilenetv3_lm PL2 no augs
    ("../logs/2024-05-20/5/", [f"{i}" for i in range(4)], "torch"),   # repghostnet_100 PL2 no augs
]

In [ ]:
FOLD = 0 if EVAL else "fullfit_0"

EXP_FOLDERS = [
    # ROUND 1 - 
    # ("../logs/2024-05-16/12/", [f"{i}" for i in range(4)], "torch"),  # vit-b0 PL2 no augs              0.71
    # ("../logs/2024-05-17/2/", [f"{i}" for i in range(4)], "torch"),   # mobilenetv2 PL2 no augs         0.69
    # ("../logs/2024-05-17/3/", [f"{i}" for i in range(4)], "torch"),   # mnasnet PL2 no augs             0.70

    # ("../logs/2024-05-18/1/", [f"{i}" for i in range(4)], "torch"),   # effvitm3 PL2 no augs            0.70
    # ("../logs/2024-05-18/8/", [f"{i}" for i in range(4)], "torch"),   # mobilenetv3_s PL2 no augs       0.??
    # ("../logs/2024-05-19/0/", [f"{i}" for i in range(4)], "torch"),   # efficientnet_lite0 PL2 no augs  0.70
    # ("../logs/2024-05-19/1/", [f"{i}" for i in range(4)], "torch"),   # effvitm4 PL2 no augs            0.??

    ("../logs/2024-05-19/9/", [f"{i}" for i in range(4)], "torch"),   # regnety_002 PL2 no augs
    # ("../logs/2024-05-19/10/", [f"{i}" for i in range(4)], "torch"),  # regnety_004 PL2 no augs
    # ("../logs/2024-05-20/1/", [f"{i}" for i in range(4)], "torch"),   # lcnet_100 PL2 no augs
    ("../logs/2024-05-20/2/", [f"{i}" for i in range(4)], "torch"),   # mobilenetv3_lm PL2 no augs
    # ("../logs/2024-05-20/5/", [f"{i}" for i in range(4)], "torch"),   # repghostnet_100 PL2 no augs
]

In [ ]:
# from util.logger import upload_to_kaggle

# upload_to_kaggle(
#     [f[0] for f in EXP_FOLDERS],
#     directory="../output/dataset_1/",
#     dataset_name="BirdCLEF 2024 Weights 1",
#     update_folders=True
# )

### Preprocessing

In [ ]:
if EVAL:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*/*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

    folds = pd.read_csv('../input/folds_4.csv')
    folds['id'] = folds['filename'].apply(lambda x: x.split('/')[-1][:-4])
    df = df.merge(folds)
    df = df[df['fold'] == 0].reset_index(drop=True)

    df["primary_label"] = df["path"].apply(lambda x:  x.split('/')[-2])

    cts = df["primary_label"].value_counts()
    low_rep = list(cts[cts < 50].index)
    df = df[df["primary_label"].isin(low_rep)].reset_index(drop=True)
else:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])
    
    # df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))
    # df = df[df["duration"] == 240].reset_index(drop=True)

### Models

In [ ]:
models = []
for e in EXP_FOLDERS:
    try:
        exp_folder, folds, runtime = e
    except:
        exp_folder, folds = e
        runtime = "torch"
    
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    for fold in folds:
        weights = exp_folder + f"{config.name}_{fold}.pt"

        model = define_model(
            config.name,
            config.melspec_config,
            head=config.head,
            aug_config=config.aug_config,
            num_classes=config.num_classes,
            n_channels=config.n_channels,
            drop_rate=config.drop_rate,
            drop_path_rate=config.drop_path_rate,
            norm=config.norm if hasattr(config, "norm") else "min_max",
            top_db=config.top_db if hasattr(config, "top_db") else None,
            exportable=config.exportable,
            verbose=True,
            pretrained=False
        )
        model = model.to(DEVICE).eval()

        model = load_model_weights(model, weights, verbose=config.local_rank == 0)
        
        models.append((model, runtime, (e[0], fold)))

        if EVAL:
            break

### Export

In [ ]:
sessions = [None for _ in range(len(models))]
if any([runtime != "torch" for _, runtime, _ in models]):
    sessions = []

    import onnx
    import onnxruntime as ort
    from onnxconverter_common import float16

    input_names = ['x']
    output_names = ['output']

    input_tensor = torch.randn(
        1 if EVAL else BATCH_SIZE,
        config.n_channels,
        config.melspec_config['n_mels'],
        313 if config.melspec_config['hop_length'] == 512 else 224
    )

    for i, (model, runtime, name) in enumerate(models):
        name = f"model_{i}.onnx"
        torch.onnx.export(
            model.encoder.cpu(),
            input_tensor,
            name,
            verbose=False,
            input_names=input_names,
            output_names=output_names,
        )
        onnx_model = onnx.load(name)
        # onnx_model = float16.convert_float_to_float16(onnx_model)
        # onnx.save(onnx_model, f"model_{i}.onnx")
        onnx.checker.check_model(onnx_model)
        ort_session = ort.InferenceSession(f"model_{i}.onnx")

        if runtime == "onnx":
            sessions.append(ort_session)
            print(f'- Convert model {name} to onnx')

        elif runtime == "openvino":
            import openvino.runtime as ov

            !mo --input_model $name # --compress_to_fp16=False
            
            core = ov.Core()
            openvino_model = core.read_model(model='model_0.xml')
            compiled_model = core.compile_model(openvino_model, device_name="CPU")
            infer_request = compiled_model.create_infer_request()
            sessions.append(infer_request)

            print(f'- Convert model {name} to openvino')
        else:
            sessions.append(None)

### Main

In [ ]:
try:
    batches = np.array_split(np.arange(len(df)), len(df) / 100)
except:
    batches = [np.arange(len(df))]

In [ ]:
inference_rows = {i : [] for i in range(len(models))}

for i, batch in enumerate(tqdm(batches)):
    df_batch = df.iloc[batch].reset_index(drop=True)

    waves = joblib.Parallel(n_jobs=4)(
        joblib.delayed(load_sample)(
            path, evaluate=EVAL, sr=SR, duration=DURATION, normalize=config.wav_norm
        )
        for path in df_batch["path"].values
    )

    for model_idx in range(len(models)):
        all_preds = [
            infer_sample(
                wave,
                [models[model_idx][:2]],
                sessions,
                device=DEVICE,
                use_fp16=USE_FP16,
            )
            for wave in waves
        ]

        for idx in range(len(df_batch)):
            y_pred = all_preds[idx]
            preds = expit(y_pred).mean(0)

            for t, pred in enumerate(preds):
                predictions = dict([(l, p) for l, p in zip(CLASSES, pred)])
                inference_rows[model_idx].append(
                    {"row_id": f"{df_batch.id[idx]}_{(t + 1) * 5}"} | predictions
                )

        del all_preds
        gc.collect()
    
    del waves
    gc.collect()

    # break

subs = {}
for model_idx in range(len(models)):
    sub = pd.DataFrame(inference_rows[model_idx])
    name = models[model_idx][2][0] + f'pl_preds_{models[model_idx][2][1]}'

    if not EVAL:
        sub[["row_id"]].to_csv(name + ".csv", index=False)
        np.save(name + ".npy", sub[CLASSES].values)

        print(f"-> Saved predictions to {name}[.csv/.npy]")
        display(sub.head(2))

        del (sub, inference_rows[model_idx])
    else:
        subs[name] = sub
    gc.collect()

In [ ]:
if EVAL:
    for k in subs.keys():
        
        exp = k.rsplit('/', 1)[0]
        config = Config(json.load(open(exp + "/config.json", "r")))
        model = config.name

        print(f"\n -> Model {exp} - {model} - PL {config.use_pl} \n")

        sub = subs[k]
        preds = sub[CLASSES].values
        max_ = preds.max(1)
        auc, aucs = macro_auc(
            df["primary_label"].values.tolist()[: len(preds)],
            preds,
            return_per_class=True
        )

        print(f"Fold 0 AUC: {auc:.5f}\n")

        df_auc = pd.DataFrame({"auc": np.clip(list(aucs.values()), 0.5, 1), "max": preds.max(0)}, index=aucs.keys())
        df_auc["count"] = df_auc.index.map(cts)

        df_auc = df_auc[df_auc.index.isin(low_rep)]

        df_auc = df_auc.sort_values('auc').head(50)
        display(df_auc.T)

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        sns.histplot(df_auc['auc'], bins=25, kde=True)
        plt.xlim(0.5, 1)
        plt.subplot(1, 3, 2)
        sns.histplot(df_auc['max'], bins=25, kde=True)
        plt.xlim(0., 1)

        plt.subplot(1, 3, 3)

        plt.scatter(df_auc["max"], df_auc["auc"], s=5)
        plt.plot([-1, 2], [0, 1.5], c="salmon")
        plt.xlim(-0.01, 1.01)
        plt.ylim(0.49, 1.01)

        plt.show()
        # break

Done !